In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from pathlib import Path
import sys
sys.path.append('..')
sys.path.append('../..')
###
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
###
from bs4 import BeautifulSoup
import re
###
import spacy
# nlp = spacy.load('en_core_web_sm', disable=["ner"])
###
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
SEED = 1
SAMPLE_FRAC = 0.1
random.seed(SEED)

In [3]:
### SST-2 ###
def parse_line(line):
    index, sent = line.split('\t')
    if 'sentence_index' in index:
        return (-1,'')
    sent = re.sub('\n', '', sent)
    index = int(index) - 1
    return (index, sent)

def get_original_sst2():
    # Load SST-2
    sst_dir = data_dir / 'SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank'
    fp = sst_dir / 'datasetSentences.txt'
    sents = {}
    with fp.open('r') as file:
        for i, line in enumerate(file):
            index, sent = parse_line(line)
            if not (index < 0):
                sents[index] = sent
    return sents
            
### IMDB Processing ###
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [5]:
data_dir = Path('../../data')
list(data_dir.iterdir())

[PosixPath('../../data/.DS_Store'),
 PosixPath('../../data/SST2-Data'),
 PosixPath('../../data/IMDB Dataset.csv')]

In [6]:
# # Load IMDB data
# df = pd.read_csv(data_dir/'IMDB Dataset.csv')
# print(f"Loaded {len(df)} samples, randomly sampling {int(SAMPLE_FRAC * len(df))} rows")
# # Sample percentage of data
# df = df.sample(frac=SAMPLE_FRAC, random_state=SEED)
# # Convert sentiment columns to numerical values
# df.sentiment = df.sentiment.apply(lambda x: 1 if x=='positive' else 0)
# df['review']=df['review'].apply(denoise_text)
# df.head(3)

In [7]:
# Load SST-2 and subsample
sents = get_original_sst2()
# num_samples = int(SAMPLE_FRAC * len(sents))        
# print(f"Got {len(sents)} samples, randomly sampling {num_samples} samples")
# sents = random.sample(sents, num_samples)
len(sents), sents[1]

(11855,
 "The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth .")

In [8]:
# # Tokenize reviews
# tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
# tokenized_texts = [tokenizer(seq) for seq in df.review]

In [9]:
# # Process reviews for sentences 
# docs = []
# for doc in nlp.pipe(df.review):
#     docs.append(doc)

In [14]:
# Tokenize SST-2 Sentences
# tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
# sents_tokenized = [tokenizer(sent) for sent in sents]

from nltk.tokenize import word_tokenize
# sents_tokenized = list(map(word_tokenize, sents))
# sents_tokenized[0]

In [15]:
from TextGenerationEvaluationMetrics import multiset_distances as MSD
from DataAugmentation.data import augmentation
from eda_nlp.code.eda import get_only_chars
# from DataAugmentation.data import back_translation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Sameer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Sameer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Sameer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /Users/Sameer/nltk_data...
[nltk_data]   Package words is already up-to-date!
../../eda_nlp/code/eda.py:177: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not '']


In [ ]:
# from inspect import getmembers, isfunction
# print(getmembers(augmentation, isfunction))

In [17]:
'''
Generating eda samples:
python eda_nlp/code/augment.py 
    --input=data/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSentences.txt 
    --output=./sst2_augmented.txt 
    --num_aug=5 --alpha_sr=0.3 --alpha_rd=0.1 --alpha_ri=0.1 --alpha_rs=0.0
'''
def get_augmented_sst2():
    fp = Path('../../sst2_augmented.txt')
    sents = {}
    with fp.open('r') as file:
        for i, line in enumerate(file):
            index, sent = parse_line(line)
            if not (index < 0):
                if index in sents:
                    sents[index].append(sent)
                else:
                    sents[index] = [sent]
    return sents

orig_sents = get_original_sst2()
orig_sents = {j:get_only_chars(t) for j, t in orig_sents.items()}
aug_sents = get_augmented_sst2()
len(orig_sents), len(aug_sents)

(11855, 11855)

In [18]:
orig_sents[0], aug_sents[0]

('the rock is destined to be the st century s new conan and that he s going to make a splash even greater than arnold schwarzenegger jean claud van damme or steven segal ',
 ['the shake is bound to be the st hundred s newly conan and that he s going to make a splash even great than matthew arnold schwarzenegger blue jean claud caravan damme or steven george segal',
  'the sway is destined to be the st c s new conan and that he s live on to make believe a splash yet nifty than benedict arnold schwarzenegger jean claud new wave damme or steven george segal',
  'the destined to be the st century s new conan and that he s going to make a splash greater than arnold schwarzenegger jean claud van damme steven segal',
  'the rock is destined to be george segal the st century s new conan and that he s going to make a splash even matthew arnold greater than arnold schwarzenegger jean claud van damme or steven atomic number segal',
  'the rock is destined to be the st century s new conan and that

In [32]:
index = 1

# ref1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that', 'ensures', 'that', 'the', 'military', 'will', 'forever', 'heed', 'Party', 'commands']
# ref2 = ['It', 'is', 'the', 'guiding', 'principle', 'which', 'guarantees', 'the', 'military', 'forces', 'always', 'being', 'under', 'the', 'command', 'of', 'the', 'Party']
# ref3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the', 'army', 'always', 'to', 'heed', 'the', 'directions', 'of', 'the', 'party']
# sen1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which', 'ensures', 'that', 'the', 'military', 'always', 'obeys', 'the', 'commands', 'of', 'the', 'party']
# sen2 = ['he', 'read', 'the', 'book', 'because', 'he', 'was', 'interested', 'in', 'world', 'history']

references = map(word_tokenize, aug_sents[index])
sentences = map(word_tokenize, [orig_sents[index]])
sentences, references = map(list, (sentences, references))

msd = MSD.MultisetDistances(references=references, min_n=1, max_n=5)
msj_distance = msd.get_jaccard_score(sentences=sentences)
msj_distance


Jaccard distances preprocess upto 5!


{1: 0.0233949945593036, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0}

In [29]:
[' '.join(s) for s in references]

['the gorgeously elaborated continuation of the lord of the anchor ring trilogy is so brobdingnagian that a tower of words can not adequately draw co writer director tool old hickory s flesh out visual sensation of joule r r tolkien s middle ground',
 'the gorgeously work out continuance of the lord of the reverberate trilogy is so brobdingnagian that a newspaper column of words can not adequately discover co author managing director peter andrew jackson s expanded vision of j radius radius tolkien s middle terra firma',
 'the gorgeously elaborate continuation of the lord of the rings trilogy is so huge that a column of words can not adequately describe co writer director peter jackson s expanded vision and then of j r r tolkien immense michael joe jackson s middle earth',
 'the gorgeously of the lord of the rings trilogy is so that a column of words can not adequately co director peter s expanded vision of j r r tolkien s middle earth',
 'the gorgeously elaborate continuation of the l

In [ ]:
' '.join(sentences[0])

In [31]:
sentences[0]

['the',
 'gorgeously',
 'elaborate',
 'continuation',
 'of',
 'the',
 'lord',
 'of',
 'the',
 'rings',
 'trilogy',
 'is',
 'so',
 'huge',
 'that',
 'a',
 'column',
 'of',
 'words',
 'can',
 'not',
 'adequately',
 'describe',
 'co',
 'writer',
 'director',
 'peter',
 'jackson',
 's',
 'expanded',
 'vision',
 'of',
 'j',
 'r',
 'r',
 'tolkien',
 's',
 'middle',
 'earth']